<a href="https://colab.research.google.com/github/dionisispriftis/CIFAR10-Autoencoder/blob/main/C4W2_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 2 Assignment: CIFAR-10 Autoencoder

Dataset: [CIFAR10](https://www.tensorflow.org/datasets/catalog/cifar10)  

## Imports

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import tensorflow_datasets as tfds

from keras.models import Sequential

## Load and prepare the dataset

The [CIFAR 10](https://www.tensorflow.org/datasets/catalog/cifar10) dataset already has train and test splits and you can use those in this exercise. Here are the general steps:

* Load the train/test split from TFDS. Set `as_supervised` to `True` so it will be convenient to use the preprocessing function we provided.
* Normalize the pixel values to the range [0,1], then return `image, image` pairs for training instead of `image, label`. This is because you will check if the output image is successfully regenerated after going through your autoencoder.
* Shuffle and batch the train set. Batch the test set (no need to shuffle).


In [ ]:
# preprocessing function
def map_image(image, label):
  image = tf.cast(image, dtype=tf.float32)
  image = image / 255.0

  return image, image # dataset label is not used. replaced with the same image input.

# parameters
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 1024

# use tfds.load() to fetch the 'train' split of CIFAR-10
train_dataset = tfds.load(name='cifar10', split='train', as_supervised=True)

# preprocess the dataset with the `map_image()` function above
train_dataset = train_dataset.map(map_image)

# shuffle and batch the dataset
train_dataset = train_dataset.shuffle(1024).batch(BATCH_SIZE)


# use tfds.load() to fetch the 'test' split of CIFAR-10
test_dataset = tfds.load(name='cifar10', split='test', as_supervised=True)

# preprocess the dataset with the `map_image()` function above
test_dataset = test_dataset.map(map_image)

# batch the dataset
test_dataset = test_dataset.batch(BATCH_SIZE)


## Build the Model

Create the autoencoder model. We'll downsample the image in the encoder layers then upsample it in the decoder path. Note that the output layer should be the same dimensions as the original image. Input images have the shape `(32, 32, 3)`. 


In [ ]:
from keras.layers import Conv2D, UpSampling2D

input = tf.keras.layers.Input(shape=(32,32,3,))

# encoder
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation = 'relu', padding='same')(input)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation = 'relu', padding='same')(x)

encoder_output = tf.keras.layers.MaxPooling2D()(x)
encoder_visualization = tf.keras.layers.Conv2D(filters=3, kernel_size=(3,3), activation = 'sigmoid', padding='same')(encoder_output)

# decoder

x = tf.keras.layers.UpSampling2D()(encoder_output)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation = 'relu', padding='same')(x)
x = tf.keras.layers.UpSampling2D()(x)

decoder_output = tf.keras.layers.Conv2D(filters=3, kernel_size=(3,3), activation = 'sigmoid', padding='same')(x)

encoder_model = tf.keras.Model(inputs = input, outputs = encoder_visualization)
model = tf.keras.Model(inputs = input, outputs = decoder_output)

model.summary()

## Configure training parameters



In [ ]:
model.compile(optimizer='adam', metrics=['accuracy'], loss='mean_squared_error')

## Training


In [ ]:
# parameters (feel free to change this)
train_steps = len(train_dataset) // BATCH_SIZE 
val_steps = len(test_dataset) // BATCH_SIZE

EPOCHS = 15

model.fit(train_dataset, epochs=EPOCHS, validation_data=test_dataset)


## Model evaluation

You can use this code to test your model locally before uploading to the grader. 

In [ ]:
result = model.evaluate(test_dataset, steps=10)

In [ ]:
test_images = test_dataset.take(1)
encoder_images = encoder_model.predict(test_images)
decoder_images = model.predict(test_images)

In [ ]:
input_images = []
for image in test_images:
  input_images.append(image[0])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
examples = 10
idx = np.random.randint(0,len(decoder_images)-1,size = examples)

fig, ax = plt.subplots(3, examples, figsize = (20,6))

for i in range(examples):
  ax[0,i].imshow(input_images[0][idx[i]])
  ax[1,i].imshow(encoder_images[idx[i]])
  ax[2,i].imshow(decoder_images[idx[i]])

In [ ]:
encoder_model.summary()

## Save your model



In [ ]:
#model.save('mymodel.h5')